<a href="https://colab.research.google.com/gist/qbeer/545fa2d88e7541f81a137f6d0363e6c9/hw7_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.) Linear SVC in case of linear separation

* load the Iris dataset (can be found in sklearn API)
* scale the data and plot the petal length vs petal width in a scatterplot colored with the target
* train an SVC model with linear kernel with default parameter settings, but once with C=1 and then C=1000
* visualize the model's decision boundary and the margins based on the coefficients learnt by the model
* interpret the results, what is the role of the C hyperparameter?

## 2.) Linear SVC but non-linear separation

* create a dataset with the following: X, y = sklearn.datasets.make_moons(noise=0.1, random_state=0)
* perform the same steps just as in the previous exercise and use the linear kernel for the SVC
* since linear SVC cannot do non-linear separation, you will need to do some workaround, for example adding polynomial features (3rd order would be a good choice)
* write down with your own words in few sentences how the support vector machine works

## 3.) Load the dataset from 2 weeks ago and build/evaluate the SVC with default settings

Reminder:

* you need to build a classifier that predicts the probability of a sample coming from a cancerous (tumor type is normal or not) person based on the measured protein levels

* train the SVM classifier (SVC in sklearn API) on every second sample (not first 50% of the data (!), use every second line)

* generate prediction for the samples that were not used during the training

To-do now:

* build default SVC, but set it to predict probabilities
* plot the ROC curve and calculate the confusion matrix for the predictions
* do the same for the CancerSEEK predictions and compare your model's performance to CancerSEEK performance (as a reference, plot it on the same figure)
*how good is the performance of the new model?

## 4.) Scale data and try different kernels

* scale your data before applying the SVC model
* plot the ROC curve and calculate the confusion matrix for the predictions
* does your model perform better or worse after scaling?
* try out other kernels (linear, poly) and evaluate the performance of the model the same way

## 5.) Split the data randomly to 3 parts: 70% train, 15% validation, 15% test data and tune hyperparameters

* prepare data as described in the title, then scale all inputs based on the training set
* select your best performing SVC model from the previous exercise
* check the behaviour of the SVC by modifying at least 3 of its hyperparameters (C, gamma, ...) and plot the AUC value vs the modified parameter (logscale may be better for visualization)
* create plots (at least 2) that shows the train, val and test accuracy based on a given hyperparameter's different values. Is it a good idea to rely on validation data when tuning hyperparameter in this case?
* select the best settings, train the SVC and evaluate with reference to CancerSEEK results with the ROC curve and the confusion matrix (match your results with CancerSEEK's results on the same dataset splitting)